In [5]:
import os
import json
import pandas as pd
import traceback

In [79]:
from langchain.chat_models import ChatOpenAI

In [28]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/home/my/Desktop/MCQGEN/.env")
load_dotenv(env_path)

True

In [59]:
KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("OPENAI_API_BASE")

In [60]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

In [63]:
from openai import OpenAI

client = OpenAI(
    base_url=BASE_URL,
    api_key=KEY
)


In [64]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [65]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    }    
}

In [66]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [67]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [68]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [69]:
TEMPLATE2="""
You are an expert english gramarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to avaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.\
If the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz.
"""

In [89]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"],
                                        template=TEMPLATE)

In [90]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [91]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                          output_variables=["quiz", "review"], verbose=True)

In [92]:
file_path = r"/home/my/Desktop/MCQGEN/data.txt"

In [93]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [94]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [95]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"
 

In [96]:
with get_openai_callback() as cb:
    response= generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From a theoretical view

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From

In [97]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")




Total Tokens: 1640
Prompt Tokens: 866
Completion Tokens: 774
Total Cost: 0.002847


In [98]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]\n\nFrom a theoretical viewpoint, probably

In [100]:
quiz = response.get('quiz')

In [102]:
quiz = json.loads(quiz)

In [104]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [105]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A field of study in artificial intelligence concerned with developing and studying statistical algorithms | b: A field of study in robotics concerned with building physical machines | c: A field of study in astronomy concerned with studying stars and planets | d: A field of study in psychology concerned with human behavior',
  'Correct': 'a'},
 {'MCQ': 'Which subdiscipline in machine learning has allowed neural networks to surpass many previous approaches?',
  'Choices': 'a: Deep learning | b: Natural language processing | c: Data mining | d: Statistics',
  'Correct': 'a'},
 {'MCQ': 'What is the application of machine learning to business problems known as?',
  'Choices': 'a: Natural language processing | b: Computer vision | c: Predictive analytics | d: Speech recognition',
  'Correct': 'c'},
 {'MCQ': 'What comprises the foundations of machine learning?',
  'Choices': 'a: Data mining | b: Mathematical optimisation | c: Email filte

In [107]:
quiz = pd.DataFrame(quiz_table_data)

In [108]:
quiz.to_csv("machinelearning.csv", index=False)